In [90]:
import pandas as pd
import nltk
import ast
import numpy as np
import os

#nltk.download('stopwords') #Only needed if exception thrown
#nltk.download('punkt')#Only needed if exception thrown
from nltk.corpus import stopwords
#from nltk.tokenize import word_tokenize 

#Using a GLobal Variable to Filter Stop Words
stop_words = set(stopwords.words('english')) 

import contractions 
import itertools

from nltk.tokenize import TweetTokenizer
tknzr = TweetTokenizer()

#from nltk import PorterStemmer()
#powerstemmer = 

In [91]:
# start Spark Session
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext

app_name = "capstone_eda"
master = "local[*]"
spark = SparkSession\
        .builder\
        .appName(app_name)\
        .master(master)\
        .getOrCreate()
sc = spark.sparkContext
sqlCtx = SQLContext(sc)

In [92]:
bad_phrases = [
                "I'm a CareerVillage staff member and I'm posting this because we know that many young people are looking for the answer to this question. This is among the most popular questions searched by youth, and we're hoping you will take a moment to share your response to it. Thank you!",
                "Things you can consider for this specific question..."
              ]


def cosine(line):
    """Compute similarity scores"""
    (doc1, n1), (doc2, n2) = ast.literal_eval(line[0])
    total = int(line[1])
    cosine = total / np.sqrt(int(n1)*int(n2))
    yield (doc1,doc2), cosine

def makeCompositeKey(inverted_index):
    """Loop over postings and yield pairs."""
    word, postings = inverted_index
    # taking advantage of symmetry, output only (a,b), but not (b,a)
    for subset in itertools.combinations(sorted(postings), 2):
        yield (str(subset), 1)
    
def create_multi_grams(sentences,bad_phrases=bad_phrases,gram_size=[1,2]):
    """
        Removes Stop Words / I / Punctuation 
        
        Input:
            Sentence: String 
        
        Ouput:
            Set of Unique Words in input
    """
    for i in bad_phrases: 
        sentences = sentences.replace(i,'')
    sentences = nltk.sent_tokenize(sentences)
    grams = []
    for sentence in sentences:
        sentence = contractions.fix(sentence)
        word_tokens = tknzr.tokenize(sentence) 
        filtered_sentence = [w.lower() for w in word_tokens if not w in stop_words and w.isalpha() and w.lower()!='i'] 
        for i in gram_size:
            ngrams = zip(*[filtered_sentence[i:] for i in range(i)])
            grams += [" ".join(ngram) for ngram in ngrams]
    return list(set(grams))

In [94]:
def splitWordsV3(pair):
    """Tokenize each question and emit postings."""
    #question,text,words = pair
    #try:
    question, text,_ = pair
    words = ast.literal_eval(text)
    #document_total = len(words*1)+len(tags)*3
    document_total = 0 
    for w in words:
        if w in cd.value.value.values:
            document_total+=cd.value[cd.value.value == w].value.index[0]
        #else:
        #    document_total+=1
            
    for w in words:
        if w in cd.value.value.values:
            yield (w, [(question,document_total,cd.value[cd.value.value == w].value.index[0])])
                
        #else:
        #    yield (w, [(question,document_total,1)])
    #except:
    #    pass

In [95]:
def print_results(data_frame,number_to_print=5,metric_type='Jaccard',filter_function=filter_stop_words):
    for i in range(number_to_print):
        Q1 = int(data_frame[i][0][0])
        Q2 = int(data_frame[i][0][1])
        Score = data_frame[i][1]
        print(metric_type+' Similarity Ranking: '+str(i+1)+' Score: {:0.05f}'.format(Score))
        print('\n') 
        print("Question 1")
        print(questions_data_df.full_question[Q1])
        print('\n') 
        print(filter_function(questions_data_df.full_question[Q1]))
        print('\n')
        print("Question 2")
        print(questions_data_df.full_question[Q2])
        print('\n')
        print(filter_function(questions_data_df.full_question[Q2]))
        print('__________________________________________________')
        print('\n')

## Load Questions Data

In [96]:
questions_data = './data/questions.csv'
questions_data_df = pd.read_csv(questions_data)
questions_data_df['full_question'] = questions_data_df.questions_title+' '+questions_data_df.questions_body
questions_data_df.head()

,questions_id,questions_author_id,questions_date_added,questions_title,questions_body,full_question
0,332a511f1569444485cf7a7a556a5e54,8f6f374ffd834d258ab69d376dd998f5,2016-04-26 11:14:26 UTC+0000,Teacher career question,What is a maths teacher? what is a ma...,Teacher career question What is a maths...
1,eb80205482e4424cad8f16bc25aa2d9c,acccbda28edd4362ab03fb8b6fd2d67b,2016-05-20 16:48:25 UTC+0000,I want to become an army officer. What can I d...,I am Priyanka from Bangalore . Now am in 10th ...,I want to become an army officer. What can I d...
2,4ec31632938a40b98909416bdd0decff,f2c179a563024ccc927399ce529094b5,2017-02-08 19:13:38 UTC+0000,Will going abroad for your first job increase ...,I'm planning on going abroad for my first job....,Will going abroad for your first job increase ...
3,2f6a9a99d9b24e5baa50d40d0ba50a75,2c30ffba444e40eabb4583b55233a5a4,2017-09-01 14:05:32 UTC+0000,To become a specialist in business management...,i hear business management is a hard way to ge...,To become a specialist in business management...
4,5af8880460c141dbb02971a1a8369529,aa9eb1a2ab184ebbb00dc01ab663428a,2017-09-01 02:36:54 UTC+0000,Are there any scholarships out there for stude...,I'm trying to find scholarships for first year...,Are there any scholarships out there for stude...


### Load Corpus Data

In [100]:
original_corpus_data = pd.read_csv('./data/All_Career_Clusters.csv')
corpus_data = pd.read_csv("./data/All_Career_Clusters_cleaned.csv")
corpus_data = corpus_data.reset_index()
corpus_data.columns = ['index','value']
#cd = sc.broadcast(corpus_data)

In [105]:
original_corpus_data.shape[0]

1109

In [106]:
occupations = set(corpus_data.value.values)
job_list=[]
for i in questions_data_df.index:
    text = questions_data_df.questions_body.iloc[i]
    for job in occupations:
        if job.lower() in text:
            #print(questions_data_df.questions_id.iloc[i]," ",job)
            job_list.append(job)
            
print("Total BLS List of Careers:" +str(original_corpus_data.shape[0]))
print('\n')
print('Total Distinct Jobs from BLS after filtering:'+str(corpus_data.shape[0]))
print('\n')
print("Total Distinct Jobs Found in CareerVillage Corpus: "+str(len(set(job_list))))
print('\n')
print(set(job_list))

Total BLS List of Careers:1109


Total Distinct Jobs from BLS after filtering:1833


Total Distinct Jobs Found in CareerVillage Corpus: 280


{'Farmer', 'Nanny', 'Cosmetologist', 'Automotive Engineer', 'Anthropologist', 'Street Vendor', 'Probation Officer', 'Wildlife Biologist', 'Urban Planner', 'Advertising', 'Physician Assistant', 'Athlete', 'Microbiologist', 'Sales Engineer', 'Poet', 'Genetic Counselor', 'Cook', 'Chemist', 'Tax Preparer', 'Fraud Examiner', 'Mortician', 'Computer Scientist', 'Boilermaker', 'Medical Technologist', 'Instructional Designer', 'Retail Sales', 'Family Therapist', 'Game Warden', 'Auditor', 'Marriage Therapist', 'Industrial-Organizational Psychologist', 'Social Worker', 'Industrial Designer', 'Computer Hardware Engineer', 'Agent', 'Prosthetist', 'Sociologist', 'Librarian', 'High School Teacher', 'Urologist', 'Statistician', 'Administrative Assistant', 'Medical Scientist', 'Janitor', 'Pharmacist', 'Biomedical Engineer', 'Registered Nurse', 'Scout', 'Marine En

In [51]:
questions_data_df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23931 entries, 0 to 23930
Data columns (total 6 columns):
questions_id            23931 non-null object
questions_author_id     23931 non-null object
questions_date_added    23931 non-null object
questions_title         23931 non-null object
questions_body          23931 non-null object
full_question           23931 non-null object
dtypes: object(6)
memory usage: 1.1+ MB


In [54]:
cols_list = [
             'questions_id',
             'questions_author_id',
             'questions_date_added',
             'questions_title',
             'questions_body',
             'full_question'
            ]

questions_data_df[cols_list] = questions_data_df[cols_list].astype(str)

In [57]:
sdf = sqlCtx.createDataFrame(questions_data_df)

In [59]:
sdf.registerTempTable("CareerVillage")

In [84]:
career = 'High School Teacher'
sql_str = "select questions_id, full_question from CareerVillage where lower(full_question) like '%{0:}%'".format(career.lower()) 

data  = sqlCtx.sql(sql_str).toPandas()

In [86]:
data.full_question.values

array(['What do I major in to become a secondary teacher? I want to be a high school teacher. #secondary-education',
       'Is there a difference between High School teachers and professors in college? #courses #personality #college #high-school ',
       'Is it better to be an elementary teacher or high school teacher? I love young kids, and if I were an art teacher, I would get to see kids exploring art for the first time. If I as a high school teacher, however, I would get to see a student for from 0 to 100 in their progress as a growing artist. \n#art-teacher #educators #teaching ',
       'Should you strive for a high paying occupation or what you like to do? I really want to be a high school teacher, but my mom always tells me how they do not make enough money, and to go for something that pays more like a doctor. I have no interest in being in the medical field or law. What should I do?\r\n\r\n#teacher #doctor #career #money',
       'How do you figure out what grade level you 